In [ ]:
# gnn_gcn_cora.py
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.transforms import NormalizeFeatures

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 1) Tải dữ liệu Cora (đã có sẵn train/val/test mask)
dataset = Planetoid(root="./data", name="Cora", transform=NormalizeFeatures())
data = dataset[0].to(DEVICE)  # 1 đồ thị duy nhất

# 2) Mô hình GCN
class GCN(torch.nn.Module):
    def __init__(self, in_dim, hidden, out_dim, dropout=0.5):
        super().__init__()
        self.conv1 = GCNConv(in_dim, hidden)
        self.conv2 = GCNConv(hidden, out_dim)
        self.dropout = dropout

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index)
        return x

model = GCN(in_dim=dataset.num_features, hidden=16, out_dim=dataset.num_classes).to(DEVICE)
opt = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()

# 3) Train/Eval
def train():
    model.train()
    opt.zero_grad()
    out = model(data.x, data.edge_index)              # logits [num_nodes, num_classes]
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    opt.step()
    return float(loss.item())

@torch.no_grad()
def test():
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=-1)

    def acc(mask):
        return (pred[mask] == data.y[mask]).float().mean().item()

    return acc(data.train_mask), acc(data.val_mask), acc(data.test_mask)

for epoch in range(1, 201):
    loss = train()
    if epoch % 20 == 0 or epoch == 1:
        tr, va, te = test()
        print(f"[{epoch:03d}] loss={loss:.4f} | acc: train={tr:.3f} val={va:.3f} test={te:.3f}")
